# Hidden Markov Model project
## Machine Learning Fundamentals

|First name|Last name|Master program|Contribution|
|----------|---------|--------------|-------------|
|BEGGARI|Islem|MLDM|25%|
|BENGUEZZOU|Idriss|DSC|25%|
|BOUABID|Iness|DSC|25%|
|MEZIANE|Ghilas|DSC|25%|

# WARNING : you need version 0.2.6 of hmmlearn

In [ ]:
# !pip install --force-reinstall hmmlearn == 0.2.6
import hmmlearn
from hmmlearn import hmm

# WARNING : check that the hmmlearn installed is version 0.2.6

In [ ]:
!pip show hmmlearn  # check that the version installed is 0.2.6

# imports

In [ ]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from tqdm import tqdm
from matplotlib import pyplot as plt  # show graph

from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, \
    f1_score, roc_auc_score

## In this notebook we will look at the NER dataset and use it to understand HMM and also construct a POS tagger at the same time.

### Data Description:
#### sentence: this column donates to which sentence the word belongs
#### Word: the word in the sentence
#### POS: Associated POS tag for the word

## Load the data

If you imported the dataset "NER dataset.csv" to you google drive, you can use the following to mount and import it.

In [ ]:
# from google.colab import drive

# drive.mount('/content/drive')

# data = pd.read_csv("/content/drive/MyDrive/MLF_project/data/NER dataset.csv", encoding='latin1')

Otherwise, if you are working locally or if you just uploaded the dataset for this session on the drive, you can use the following to import it.

In [ ]:
data = pd.read_csv("./data/NER dataset.csv", encoding='latin1')

In [ ]:
data = data.fillna(method="ffill")
data = data.rename(columns={'Sentence #': 'sentence'})
data.head(5)

# Data pre-processing
If you want to do some pre-processing (lowercase any words, remove stop words, replace numbers/names by a unique NUM/NAME token, etc.) you can do it here in the pipeline.

Note : you could create a new dataset `data_pre_precessed = pre_process(data)` to keep both version and compare the effect of you pre-processing.

In [ ]:
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
import re

def pre_processing(data):
    # Convert to lower case
    data['Word'] = data['Word'].apply(lambda x: x.lower())

    # Remove punctuation and special characters
    data['Word'] = data['Word'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

    # Remove numerical values
    data['Word'] = data['Word'].apply(lambda x: re.sub(r'\d+', '', x))

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    data['Word'] = data['Word'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

    # Stemming
    # ps = PorterStemmer()
    # data['Word'] = data['Word'].apply(lambda x: ' '.join([ps.stem(word) for word in x.split()]))

    # # Lemmatization
    # lem = WordNetLemmatizer()
    # data['Word'] = data['Word'].apply(lambda x: ' '.join([lem.lemmatize(word) for word in x.split()]))

    return data

First let's collect the unique words and the unique POS tags in the dataset, we will use this to construct the HMM later

In [ ]:
data = pre_processing(data)

tags = list(set(data.POS.values))  # Unique POS tags in the dataset
words = list(set(data.Word.values))  # Unique words in the dataset
len(tags), len(words)

### We have 42 different tags and 35,178 different words, so the HMM that we construct will have the following properties
- The hidden states of the this HMM will correspond to the POS tags, so we will have 42 hidden states.
- The Observations for this HMM will correspond to the sentences and their words.

#### Before constructing the HMM, we will split the data into train and test.

In [ ]:
y = data.POS
X = data.drop('POS', axis=1)

gs = GroupShuffleSplit(n_splits=2, test_size=.33, random_state=42)
train_ix, test_ix = next(gs.split(X, y, groups=data['sentence']))

data_train = data.loc[train_ix]
data_test = data.loc[test_ix]

In [ ]:
data_train.head(5)

In [ ]:
data_test.head(5)

Now lets encode the POS and Words to be used to generate the HMM.

In [ ]:
dfupdate = data_train.sample(frac=.15, replace=False, random_state=42)
dfupdate.Word = 'UNKNOWN'
data_train.update(dfupdate)
words = list(set(data_train.Word.values))

# Convert words and tags into numbers
word2id = {w: i for i, w in enumerate(words)}
tag2id = {t: i for i, t in enumerate(tags)}
id2tag = {i: t for i, t in enumerate(tags)}
len(tags), len(words)

In your theory classes you might have seen that the Hidden Markov Models can be learned by using the Baum-Welch algorithm by just using the observations.
Although we can learn the Hidden States (POS tags) using Baum-Welch algorithm,We cannot map them back the states (words) to the POS tag. So for this exercise we will skip using the BW algorithm and directly create the HMM.

For creating the HMM we should build the following three parameters. 
- `startprob_`
- `transmat_`
- `emissionprob_`

To construct the above mentioned paramters let's first create some useful matrices that will assist us in creating the above three parameters

In [ ]:
count_tags = dict(data_train.POS.value_counts())  # Total number of POS tags in the dataset
# Now let's create the tags to words count
count_tags_to_words = data_train.groupby(['POS']).apply(
    lambda grp: grp.groupby('Word')['POS'].count().to_dict()).to_dict()
# We shall also collect the counts for the first tags in the sentence
count_init_tags = dict(data_train.groupby('sentence').first().POS.value_counts())

# Create a mapping that stores the frequency of transitions in tags to it's next tags
count_tags_to_next_tags = np.zeros((len(tags), len(tags)), dtype=int)
sentences = list(data_train.sentence)
pos = list(data_train.POS)
for i in tqdm(range(len(sentences)), position=0, leave=True):
    if (i > 0) and (sentences[i] == sentences[i - 1]):
        prevtagid = tag2id[pos[i - 1]]
        nexttagid = tag2id[pos[i]]
        count_tags_to_next_tags[prevtagid][nexttagid] += 1

Now Let's build the parameter matrices 

In [ ]:
startprob = np.zeros((len(tags),))
transmat = np.zeros((len(tags), len(tags)))
emissionprob = np.zeros((len(tags), len(words)))
num_sentences = sum(count_init_tags.values())
sum_tags_to_next_tags = np.sum(count_tags_to_next_tags, axis=1)
for tag, tagid in tqdm(tag2id.items(), position=0, leave=True):
    floatCountTag = float(count_tags.get(tag, 0))
    startprob[tagid] = count_init_tags.get(tag, 0) / num_sentences
    for word, wordid in word2id.items():
        emissionprob[tagid][wordid] = count_tags_to_words.get(tag, {}).get(word, 0) / floatCountTag
    for tag2, tagid2 in tag2id.items():
        transmat[tagid][tagid2] = count_tags_to_next_tags[tagid][tagid2] / sum_tags_to_next_tags[tagid]

## Task 1: Similar to how we built the hidden state transition probability matrix as shown above, you will built the transition probability between the words. With this matrix write a function that can calculate the log likelihood given a sentence.
---

This code block creates a transition probability matrix, which stores the probability of each word transitioning to the next word in a sentence. The input data is a set of sentences, which is split into individual words. The `word2id` dictionary is used to map each word to a unique ID. 

Then, a count matrix is created that stores the frequency of transitions between words. The count matrix is normalized to get the transition probability matrix. The resulting matrix can be used to predict the probability of a given word transitioning to the next word in a sentence. 

In [ ]:
# Create a mapping that stores the frequency of transitions in words to their next words
count_words_to_next_words = np.zeros((len(words), len(words)), dtype=int)
sentences = list(data_train.sentence)
words_list = list(data_train.Word)

for i in range(1, len(sentences)):
    if (sentences[i] == sentences[i-1]):
        prevword = words_list[i-1]
        nextword = words_list[i]
        if prevword in word2id and nextword in word2id:
            prevwordid = word2id[prevword]
            nextwordid = word2id[nextword]
            count_words_to_next_words[prevwordid][nextwordid] += 1

# Normalize the count of transitions to their next words to obtain a transition probability matrix.
word_transition_matrix = np.zeros((len(words), len(words)))
sum_words_to_next_words = np.sum(count_words_to_next_words, axis=1)

# Iterate over all words and compute the probability of transitioning to each possible next word.
# If a word has no transitions to any other words, leave its row in the transition matrix as zeros.
for i in tqdm(range(len(words)), position=0, leave=True):
    if sum_words_to_next_words[i] > 0:
        word_transition_matrix[i] = count_words_to_next_words[i] / sum_words_to_next_words[i]

This function calculates the log likelihood of a given sentence based on a word transition matrix.
It first converts the sentence to a list of word ids using a dictionary mapping of words to ids.
Then it iterates through the list of word ids, calculating the log likelihood of each transition
based on the word transition matrix. 

Finally, it returns the total log likelihood of the sentence.

In [ ]:
from typing import List

def calculate_log_likelihood(sentence: List[str], word_transition_matrix) -> float:
    """
    Given a sentence and word_transition_matrix, returns the log likelihood of the sentence.
    """
    # Convert the sentence to a list of word ids
    word_ids = [word2id.get(word, word2id['UNKNOWN']) for word in sentence]
    
    # Calculate the log likelihood using the word transition matrix
    log_likelihood = 0.0
    for i in range(1, len(word_ids)):
        prev_word_id = word_ids[i-1]
        curr_word_id = word_ids[i]
        log_likelihood += np.log(word_transition_matrix[prev_word_id, curr_word_id])
    
    return log_likelihood

We define some test given by **Mr. Richard Serrano**. 

In [ ]:
print(calculate_log_likelihood(["This", "is", "a", "protest", "about", "how", "the", "new", "law", "is", "not", "in", "the", "interest", "of", "the", "people"], word_transition_matrix)) # = -70.96
print(calculate_log_likelihood(["The","international","conference","will","continue","as","planned","on","Friday"], word_transition_matrix)) # = -37.09
print(calculate_log_likelihood(["Who", "are", "you", "?"], word_transition_matrix)) # = -15.99
print(calculate_log_likelihood(["You", "are", "not", "me"], word_transition_matrix)) # = -13.90
print(calculate_log_likelihood(["Do", "you", "expect", "to", "be", "happy", "to", "work", "late"], word_transition_matrix)) # = -35.12
print(calculate_log_likelihood(["This", "is", "a", "test", "sentence"], word_transition_matrix)) # = -inf

#### Now we will continue to constructing the HMM.

We will use the hmmlearn implementation to initialize the HMM Model

In [ ]:
model = hmm.MultinomialHMM(n_components=len(tags), algorithm='viterbi', random_state=42)
model.startprob_ = startprob
model.transmat_ = transmat
model.emissionprob_ = emissionprob

#### Before using the HMM to predict the POS tags, we have to fix the training set as some of the words and tags in the test data might not appear in the training data so we collect this data to use it later.

In [ ]:
data_test.loc[~data_test['Word'].isin(words), 'Word'] = 'UNKNOWN'
word_test = list(data_test.Word)
samples = []
for i, val in enumerate(word_test):
    samples.append([word2id[val]])

# TODO use panda solution
lengths = []
count = 0
sentences = list(data_test.sentence)
for i in tqdm(range(len(sentences)), position=0, leave=True):
    if (i > 0) and (sentences[i] == sentences[i - 1]):
        count += 1
    elif i > 0:
        lengths.append(count)
        count = 1
    else:
        count = 1

Now that we have the HMM ready lets predict the best path from them.

In [ ]:
pos_predict = model.predict(samples, lengths)
pos_predict

The hmmlearn predict function will give the best probable path for the given sentence using the Viterbi algorithm.

## Task 2: Using the model parameters (startprob_, transmat_, emissionprob_) write the viterbi algorithm from scratch to calculate the best probable path and compare it with the hmmlearn implementation.

Now before using these matrices 

In [ ]:
def Viterbi(pi: np.array, a: np.array, b: np.array, obs: List) -> np.array:
    """
    Write the viterbi algorithm from scratch to find the best probable path
    attr:
      pi: initial probabilities
      a: transition probabilities
      b: emission probabilities
      obs: list of observations
    return:
      array of the indices of the best hidden states
    """
    n = a.shape[0]
    T = len(obs)

    # Initialize the trellis table
    trellis = np.zeros((n, T))
    backpointers = np.zeros((n, T), dtype=int)

    # Initialize the first column
    trellis[:, 0] = pi * b[:, obs[0]]

    # Loop over the remaining columns
    for t in range(1, T):
        for j in range(n):
            # Calculate the scores for all possible previous states
            scores = trellis[:, t-1] * a[:, j] * b[j, obs[t]]

            # Choose the highest score
            trellis[j, t] = np.max(scores)
            backpointers[j, t] = np.argmax(scores)

    # Backtrack to find the most probable sequence of hidden states
    path = np.zeros(T, dtype=int)
    path[-1] = np.argmax(trellis[:, -1])
    for t in range(T-2, -1, -1):
        path[t] = backpointers[path[t+1], t+1]

    return path

We also define a function to decode a given path. This function will be used to decode the best path that we get from the Viterbi algorithm.

In [ ]:
def decode_path(path, hidden_states):
    return [hidden_states[state_index] for state_index in path]

Lets compare the best probable path found by the hmmlearn implementation and the one we found using the viterbi algorithm.

In [ ]:
obs = ["This", "is", "a", "protest", "about", "how", "the", "new", "law", "is", "not", "in", "the", "interest", "of", "the", "people"]
obs_idx = [word2id[word] if word in word2id else word2id['UNKNOWN'] for word in obs]

# With our Viterbi function we obtain the best probable path
path_viterbi = Viterbi(model.startprob_, model.transmat_, model.emissionprob_, obs_idx)

# Using the decode method from hmmlearn to obtain the best probable path
logprob, path_hmmlearn = model.decode(np.array(obs_idx).reshape(-1, 1))

# Compare the two paths
print(f"Viterbi path: {path_viterbi}")
print(f"hmmlearn path: {path_hmmlearn}")

### Task 3: Let's try to form our own HMM
In this task you will try to formulate your own HMM. Image a toy example that you think that closely relates to a Hidden Markov Model.

Steps:
 1. Define your hidden states
 2. Define your observable states
 3. Randomly generate your observations

Below is an example to demonstrate:

-In this toy HMM example, we have two hidden states 'healthy' and 'sick' these states relate to the state of a pet. In this example we cannot exactly know the situation of the pet if it is 'healthy' or 'sick'

-The observable states in this formulation is the what our pet is doing, whether it is sleeping, eating or pooping. We ideally want to determine if the pet is sick or not using these observable states


```python
hidden_states = ['healthy', 'sick']
observable_states = ['sleeping', 'eating', 'pooping']
observations = []
for i in range(100):
  observations.append(random.choice(observable_states))
```

TASK 3: Now try to formulate your HMM here.

In [ ]:
import random

hidden_emotions = ['happy', 'angry', 'sad', 'calm', 'disgusted']
observable_face = ['smiling', 'frowning', 'crying', 'neutral', 'grimacing']
observable_map_face = {'smiling': 0, 'frowning': 1, 'crying': 2, 'neutral': 3, 'grimacing': 4}

# Set a seed value for reproducibility
random.seed(54545324)

observations_face_emotions = []
for i in range(100):
    observations_face_emotions.append(observable_map_face[random.choice(observable_face)])


Even tough we have generated the data randomly, for the learning purposes, let's try to learn an HMM from this data. For this we have to construct the Baum-Welch algorithm from scratch. Below is the skeleton of the Baum-Welch learning algorithm.

## TASK 4: Complete the forward and backward probs functions in the Baum-Welch algorithm and try it with your formulated HMM.

In [ ]:
import numpy as np


def baum_welch(observations, observations_vocab, n_hidden_states):
    """
    Baum-Welch algorithm for estimating the HMM parameters
    :param observations: observations
    :param observations_vocab: observations vocabulary
    :param n_hidden_states: number of hidden states to estimate
    :return: a, b (transition matrix and emission matrix)
    """

    def forward_probs(observations, observations_vocab, n_hidden_states, a_, b_) -> np.array:
        """
        forward pass to calculate alpha
        :param observations: observations
        :param observations_vocab: observation vocabulary
        :param n_hidden_states: number of hidden states
        :param a_: estimated alpha
        :param b_: estimated beta
        :return: refined alpha_
        """
        a_start = 1 / n_hidden_states
        alpha_ = np.zeros((n_hidden_states, len(observations)), dtype=float)
    
        # Initialize alpha_0
        alpha_[:, 0] = a_start * b_[:, np.where(observations_vocab == observations[0])[0][0]]

        # Recursively calculate alpha_t for t > 0
        for t in range(1, len(observations)):
            for j in range(n_hidden_states):
                alpha_[j, t] = np.sum(alpha_[:, t - 1] * a_[:, j]) * b_[j, np.where(observations_vocab == observations[t])[0][0]]

        return alpha_

    def backward_probs(observations, observations_vocab, n_hidden_states, a_, b_) -> np.array:
        """
        backward pass to calculate alpha
        :param observations: observations
        :param observations_vocab: observation vocabulary
        :param n_hidden_states: number of hidden states
        :param a_: estimated alpha
        :param b_: estimated beta
        :return: refined beta_
        """
        beta_ = np.zeros((n_hidden_states, len(observations)), dtype=float)
        beta_[:, -1:] = 1

        # Recursively calculate beta_t for t < T-1
        for t in range(len(observations) - 2, -1, -1):
            for j in range(n_hidden_states):
                beta_[j, t] = np.sum(a_[j, :] * b_[:, np.where(observations_vocab == observations[t+1])[0][0]] * beta_[:, t+1])

        return beta_

    def compute_gamma(alfa, beta, observations, vocab, n_samples, a_, b_) -> np.array:
        """

        :param alfa:
        :param beta:
        :param observations:
        :param vocab:
        :param n_samples:
        :param a_:
        :param b_:
        :return:
        """
        # gamma_prob = np.zeros(n_samples, len(observations))
        gamma_prob = np.multiply(alfa, beta) / sum(np.multiply(alfa, beta))
        return gamma_prob

    def compute_sigma(alfa, beta, observations, vocab, n_samples, a_, b_) -> np.array:
        """

        :param alfa:
        :param beta:
        :param observations:
        :param vocab:
        :param n_samples:
        :param a_:
        :param b_:
        :return:
        """
        sigma_prob = np.zeros((n_samples, len(observations) - 1, n_samples), dtype=float)
        denomenator = np.multiply(alfa, beta)
        for i in range(len(observations) - 1):
            for j in range(n_samples):
                for k in range(n_samples):
                    index_in_vocab = np.where(vocab == observations[i + 1])[0][0]
                    sigma_prob[j, i, k] = (alfa[j, i] * beta[k, i + 1] * a_[j, k] * b_[k, index_in_vocab]) / sum(
                        denomenator[:, j])
        return sigma_prob

    # initialize A ,B
    a = np.ones((n_hidden_states, n_hidden_states)) / n_hidden_states
    b = np.ones((n_hidden_states, len(observations_vocab))) / len(observations_vocab)
    for iter in tqdm(range(2000), position=0, leave=True):

        # E-step caclculating sigma and gamma
        alfa_prob = forward_probs(observations, observations_vocab, n_hidden_states, a, b)  #
        beta_prob = backward_probs(observations, observations_vocab, n_hidden_states, a, b)  # , beta_val
        gamma_prob = compute_gamma(alfa_prob, beta_prob, observations, observations_vocab, n_hidden_states, a, b)
        sigma_prob = compute_sigma(alfa_prob, beta_prob, observations, observations_vocab, n_hidden_states, a, b)

        # M-step caclculating A, B matrices
        a_model = np.zeros((n_hidden_states, n_hidden_states))
        for j in range(n_hidden_states):  # calculate A-model
            for i in range(n_hidden_states):
                for t in range(len(observations) - 1):
                    a_model[j, i] = a_model[j, i] + sigma_prob[j, t, i]
                normalize_a = [sigma_prob[j, t_current, i_current] for t_current in range(len(observations) - 1) for
                               i_current in range(n_hidden_states)]
                normalize_a = sum(normalize_a)
                if normalize_a == 0:
                    a_model[j, i] = 0
                else:
                    a_model[j, i] = a_model[j, i] / normalize_a

        b_model = np.zeros((n_hidden_states, len(observations_vocab)))

        for j in range(n_hidden_states):
            for i in range(len(observations_vocab)):
                indices = [idx for idx, val in enumerate(observations) if val == observations_vocab[i]]
                numerator_b = sum(gamma_prob[j, indices])
                denominator_b = sum(gamma_prob[j, :])
                if denominator_b == 0:
                    b_model[j, i] = 0
                else:
                    b_model[j, i] = numerator_b / denominator_b

        a = a_model
        b = b_model
    return a, b

In [ ]:
hidden_states = ['healthy', 'sick']
observable_states = ['sleeping', 'eating', 'pooping']
observable_map = {'sleeping': 0, 'eating': 1, 'pooping': 2}
observations = []

for i in range(100):
    observations.append(observable_map[random.choice(observable_states)])

A, B = baum_welch(observations=observations, observations_vocab=np.array(list(observable_map.values())),
                  n_hidden_states=2)

We use our own formulated HMM to try the ``baum_welch`` algorithm. The transition matrix and emission matrix will be used in our ```test_sentence``` to predict the most probable path with the ``Viterbi`` algorithm that we implemented.

In [ ]:
transmat, emission = baum_welch(observations=observations_face_emotions, observations_vocab=np.array(list(observable_map_face.values())),
                  n_hidden_states=len(hidden_emotions))

The ``test_sentence`` function takes a sentence as input and uses the Viterbi algorithm to predict the sequence of hidden emotions that are most likely to have generated the observable facial expressions present in the sentence.

We first maps the observable facial expressions in the sentence to their corresponding indices using the ``observable_map_face`` dictionary. We then initializes the Viterbi algorithm with the start probabilities for each hidden emotion, which we set to be uniform across all emotions.

In [ ]:
def test_sentence(sentence):
    obs_idx = [observable_map_face.get(o, -1) for o in sentence]
    # we remove the -1 values (unrecognized words)
    obs_idx = [o for o in obs_idx if o != -1]

    # we run the Viterbi algorithm to find the most likely path
    n_hidden_states = len(hidden_emotions)
    startprob = np.ones(n_hidden_states) / n_hidden_states
    path_viterbi = Viterbi(startprob, transmat, emission, obs_idx)

    print("The sentence was: ", ' '.join(sentence))
    print("The Viterbi path was: ", path_viterbi)
    print("The sentiment of the sentence is: ", decode_path(path_viterbi, hidden_emotions))
    print("---")


We can test our model with the test defined below. 

NOTE: We generated the observation sequence randomly with ``observations_face_emotions.append(observable_map_face[random.choice(observable_face)])``, so the predicted emotions may not always be accurate or meaningful. Furthermore, the observable states are only ``['smiling', 'frowning', 'crying', 'neutral', 'grimacing']``

In [ ]:
# ANGRY
test_sentence(["I", "was", "frowning", "at", "the", "cat", "in", "the", "street", "."])

# HAPPY
test_sentence(["She", "was", "smiling", "at", "me", "."])
test_sentence(["He", "grinned", ",", "his", "face", "smiling", "with", "delight", "as", "he", "read", "the", "good", "news", "."]) 

# DISGUSTED
test_sentence(["She", "couldn't", "hide", "her", "grimacing", "face", "as", "she", "tasted", "the", "spicy", "food", "."])

# SAD
test_sentence(["She", "was", "crying", "because", "she", "had", "lost", "her", "dog", "."])

# CALM
test_sentence(["He", "had", "a", "neutral", "expression", "."])
